In [77]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.fftpack import fft, fftfreq
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [69]:
# Step 1: Generate complex synthetic time series data
t = np.arange(0, 20, 0.1)  # Time points from 0 to 20 with step size of 0.1
y = (5 * np.sin(2 * np.pi * t / 5) +         # Sine wave with period 5
     3 * np.cos(2 * np.pi * t / 3) +         # Cosine wave with period 3
     2 * np.sin(2 * np.pi * t / 7) +         # Sine wave with period 7
     1.5 * np.cos(2 * np.pi * t / 1.5) +     # Cosine wave with period 1.5
     np.random.normal(0, 0.5, len(t)))       # Adding random noise

In [70]:
# Step 2: Define Fourier series function with variable frequency and n_terms
def fourier_series(t, *a, dominant_freq):
    ret = a[0]  # Constant term
    n_terms = (len(a) - 1) // 2
    for i in range(1, n_terms + 1):
        ret += a[2*i-1] * np.cos(2 * np.pi * i * dominant_freq * t) + a[2*i] * np.sin(2 * np.pi * i * dominant_freq * t)
    return ret

In [75]:
# Step 3: Fit Fourier series to the generated data with adjustable n_terms, dominant_freq, and amplitude
def fit_fourier_series(n_terms, dominant_freq, amplitude):
    # Initial guess and bounds for the coefficients
    initial_guess = np.random.rand(2 * n_terms + 1)
    
    # Define a lambda function to pass the dominant_freq to the curve_fit function
    fourier_func = lambda t, *a: fourier_series(t, *a, dominant_freq=dominant_freq)
    
    # Fit the Fourier series to the generated data
    popt, _ = curve_fit(fourier_func, t, y, p0=initial_guess)
    
    # Generate predictions for a larger time range
    t_future = np.arange(0, 25, 0.1)
    
    # Apply the amplitude scaling factor to the Fourier series
    y_pred = amplitude * fourier_series(t_future, *popt, dominant_freq=dominant_freq)
    
    # Calculate the mean squared error (MSE)
    y_pred_mse = amplitude * fourier_series(t, *popt, dominant_freq=dominant_freq)
    mse = mean_squared_error(y, y_pred_mse)
    print("MSE of fitted model on train set:", round(mse, 2))
    
    # Plot the original data and the fitted Fourier series
    plt.figure(figsize=(14, 8))
    plt.scatter(t, y, label='Original Data', color='blue', s=15)
    plt.plot(t_future, y_pred, label=f'Fourier Series (n_terms={n_terms}, freq={dominant_freq:.2f}, amp={amplitude})', color='red')
    plt.axvline(x=t[-1], linestyle='--', color='green', label='Prediction Start')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title(f'Fitting Fourier Series (MSE: {mse:.4f})')
    plt.title('Fitting Fourier Series with Adjustable n_terms, Frequency, and Amplitude')
    plt.show()

In [78]:
# Step 4: Create interactive sliders for n_terms, dominant_freq, and amplitude
interact(fit_fourier_series,
         n_terms=IntSlider(min=0, max=20, step=1, value=5, description='n_terms'),
         dominant_freq=FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='dominant_freq'),
         amplitude=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='amplitude'))

interactive(children=(IntSlider(value=5, description='n_terms', max=20), FloatSlider(value=1.0, description='d…

<function __main__.fit_fourier_series(n_terms, dominant_freq, amplitude)>